## Introduction
 - We know how to get data now we focus on data itself
 - how good is it?
 - can it be trusted?
 
## What is Data Cleaning
 - iterative process - detect, correct and loop
 - problems in data
     - text where we expect to find numeric data (two instead of 2)
     - may have missing elements, extra elements or totally different structure
     - out of range numbers
     - outliers compared to standard distributions
     - date entries in different formats
     
## Sources of dirt data
- user entry errors
- poor data coding standard
- different schemas for same kind of data
- legacy systems
- evolving systems
- no unique identifiers
- lost data during data migration
- programmer error
- corruption in transmission

## Measures of data quality
- Validity: Conforms to a schema or other constraints
- Accuracy: Conforms to gold standards
- Completeness: all records?
- Consistency: matches other data
- uniformity: same units

## Blueprint for cleaning
- Audit data 
    - programmatically check data and create a report on quality of data
    - statistical analysis to check for outliers
- create a data cleaning plan
    - identify causes
    - define operations to clean
    - test to ensure operations will clean as expected
- run the plan
- possibly manually cleaning

## Cleaning using blueprint
 - We will use a sub set of open street map data for Chicago to see cleaning
 - We will initially take a sample of our OSM file as the main one is quite big. The main file for Chicago can be downloaded [here](https://mapzen.com/data/metro-extracts/metro/chicago_illinois/)

In [1]:
from collections import defaultdict

import xml.etree.cElementTree as ET
import re

def get_element(osm_file, tags=('node', 'way', 'relation')):
    """Yield element if it is the right type of tag

    Reference:
    http://stackoverflow.com/questions/3095434/inserting-newlines-in-xml-file-generated-via-xml-etree-elementtree-in-python
    """
    context = iter(ET.iterparse(osm_file, events=('start', 'end')))
    _, root = next(context)
    for event, elem in context:
        if event == 'end' and elem.tag in tags:
            yield elem
            root.clear()

In [2]:
def take_sample(k, osm_file, sample_file):
    with open(sample_file, 'wb') as output:
        output.write('<?xml version="1.0" encoding="UTF-8"?>\n')
        output.write('<osm>\n  ')

        # Write every kth top level element
        for i, element in enumerate(get_element(osm_file)):
            if i % k == 0:
                # print "i is {}".format(i)
                output.write(ET.tostring(element, encoding='utf-8'))

        output.write('</osm>')

In [3]:
#take_sample(10, "../chicago_illinois.osm", "../sample_10.osm")

Now we will audit street types

In [4]:
def print_sorted_dict(d):
    keys = d.keys()
    keys = sorted(keys, key=lambda s: s.lower())
    for k in keys:
        v = d[k]
        print "%s: %d" % (k, v) 

In [5]:
street_type_re = re.compile(r'\S+\.?$', re.IGNORECASE)
street_types = defaultdict(int)

def audit_street_type(street_types, street_name):
    m = street_type_re.search(street_name)
    if m:
        street_type = m.group()

        street_types[street_type] += 1

def is_street_name(elem):
    return (elem.tag == "tag") and (elem.attrib['k'] == "addr:street")

def audit1(osm_file):
    for event, elem in ET.iterparse(osm_file):
        if is_street_name(elem):
            audit_street_type(street_types, elem.attrib['v'])    
    print_sorted_dict(street_types)    

In [6]:
audit1("../sample_10.osm")

30: 1
38: 1
59: 2
60008: 1
83: 1
?: 3
Ave: 9
Avenue: 31504
B: 20
Belmont: 1
Blvd: 4
Boulevard: 818
Broadway: 46
C: 15
Center: 2
Cir: 11
Circle: 18
Court: 153
Ct: 27
Ct.: 1
D: 17
Dr: 61
Dr.: 1
Drive: 350
E: 21
East: 1
F: 28
G: 48
H: 52
Highway: 39
J: 62
Justamere: 1
K: 3
L: 86
Lane: 65
Ln: 8
M: 66
Market: 7
N: 62
North: 2
O: 42
Park: 15
Parkway: 132
Path: 2
Place: 2470
Plaza: 5
Rd: 8
RD: 1
Rd.: 2
road: 1
Road: 662
Row: 1
Square: 3
St: 22
st: 1
St.: 1
Street: 13765
Terrace: 55
Trail: 4
Trl: 2
Vista: 1
W: 1
West: 10


- We have taken the street types that the parser recognized. 
- we can see that there are multiple forms of each type. e.g. we have multiple forms of avenue.
- now we have street types we would decide what type of cleaning do we want. Which ones do we keep and which to rename? Do we want to?

## Auditing Validity
We may have different type of validity checks

- mandatory
- unique
- foreign key
- cross field constraints e.g. start date must be before end date
- fields should have specific type e.g. numerical 2 vs. two
- patterns
- in a range
- should be in a set

## Wikipedia Infobox Dataset
 - wikipedia has data infoboxes besides main article
 - [dbpedia](http://wiki.dbpedia.org/datasets) has taken data from wikipedia and shared in a way that we can download
 - we will use cities data set that contains information about all cities

## Auditing a Cross-Field Constrain

In [7]:
import csv

In [8]:
def get_row(input_file, skip_lines):
    rows = csv.DictReader(open(input_file))

    skipped = 0
    for row in rows:
        if skipped < skip_lines:
            skipped += 1
            continue
        
        yield row

In [9]:
def is_number(v):
    try:
        float(v)
        return True
    except:
        return False

def ensure_float(v):
    if is_number(v):
        return float(v)
    else:
        print "v is {}".format(v)

In [10]:
import math

def audit_population_denisty(input_file):
    for city in get_row(input_file, 3):
        population = ensure_float(city['populationTotal'])
        area = ensure_float(city['areaLand'])
        population_denisty = ensure_float(city['populationDensity'])
        if population and area and population_denisty:
            calculated_density = population / area
            
            if math.fabs(calculated_density - population_denisty) > 10:
                print "Possibly bad population denisty for {}".format(city['name'])

In [11]:
audit_population_denisty('cities.csv')

v is NULL
v is NULL
v is NULL
v is NULL
v is NULL
v is NULL
v is NULL
v is NULL
v is NULL
v is NULL
v is NULL
v is NULL
v is {1.01787e+08|1.019e+08}
v is NULL
v is {3.15979e+07|3.17e+07}
v is {233.4|234.789}
v is {5.51667e+07|5.53e+07}
v is {181.159|1215.3}
v is {6.36e+07|6.37137e+07}
v is {176.912|177.3}
v is {3.78138e+07|3.79e+07}
v is {504.8|507.029}
v is {2.02e+07|2.02019e+07}
v is {47.1|47.1431}
v is {2.87489e+08|2.875e+08}
v is NULL
v is {2.5355e+07|2.5356e+07}
v is NULL
v is {2.512e+08|2.51229e+08}
v is NULL
v is {8.25e+07|8.26206e+07}
v is {379.114|379.7}
v is NULL
Possibly bad population denisty for Ketchikan Alaska
v is {4.48e+06|4.48068e+06}
v is {400.388|400.4}
v is {1.458e+07|1.45816e+07}
v is {321.2|321.237}
v is {1.71198e+07|1.712e+07}
v is {666.76|666.8}
v is {2.07e+06|2.07199e+06}
v is {1103.29|1103.3}
v is {4.61e+06|4.61018e+06}
v is {642.9|642.937}
v is {3.26e+06|3.26339e+06}
v is {985.487|985.5}
v is {9.057e+07|9.05719e+07}
v is {278.882|278.9}
v is {1.274e+07|1.274

## Correcting Validity
Your task is to check the "productionStartYear" of the DBPedia autos datafile for valid values.
The following things should be done:
- check if the field "productionStartYear" contains a year
- check if the year is in range 1886-2014
- convert the value of the field to be just a year (not full datetime)
- the rest of the fields and values should stay the same
- if the value of the field is a valid year in the range as described above,
  write that line to the output_good file
- if the value of the field is not a valid year as described above,
  write that line to the output_bad file
- discard rows (neither write to good nor bad) if the URI is not from dbpedia.org
- you should use the provided way of reading and writing data (DictReader and DictWriter)
  They will take care of dealing with the header.

In [12]:
import csv
import pprint

def process_file(input_file, output_good, output_bad):

    with open(input_file, "r") as f, open(output_good, 'w') as good, open(output_bad, 'w') as bad:
        reader = csv.DictReader(f)

        header = reader.fieldnames

        good_writer = csv.DictWriter(good, delimiter=",", fieldnames=header)
        good_writer.writeheader()
        bad_writer = csv.DictWriter(bad, delimiter=",", fieldnames=header)
        bad_writer.writeheader()

        for line in reader:
            uri = line['URI']
            if 'dbpedia.org' not in uri:
                continue

            production_start_year = line['productionStartYear']
            try:
                year = int(production_start_year[0:4])
                good_writer.writerow(line)
            except ValueError:
                bad_writer.writerow(line)

In [13]:
process_file('../autos.csv', 'autos-valid.csv', 'FIXME-autos.csv')

## Auditing Accuracy
 - difficult to do as needs gold standard data

### Problems faced
- some values are arrays
- value of one column written in another column
- underscores used - regex can be used to extract the actual data
- possibly valid countries - in another language

## Auditing Completeness
 - can be difficult to audit as "You don't know what you don't know"
 - about missing records
 - need reference data

## Auditing Consistency
- two different entries should not contradict each other
- to fix this 
    - which data source do we trust more?
    - which data collection method is more reliable?

## Auditing Uniformity
- All fields using same units of measurement
- pick the type of the field and see whether it is as per your expectation or not
- print out unexpected things to see what is happening with your data

In [14]:
#counts is {'nulls': 0, 'empties': 0, 'arrays': 0}
def audit_float_field(v, counts, min_val, max_val):
    v = v.strip()
    if v == 'NULL':
        counts['nulls'] += 1
    elif v == "":
        counts['empties'] += 1
    elif is_array(v):
        counts['arrays'] += 1
    elif not is_number(v):
        print "found not number {}".format(v)
    else:
        v = float(v)
        if not (min_val <= v <= max_val):
            print "found out of range value {}".format(v)

## More About Correcting Data
- we are removing typographical errors or correcting them
- validating against known list of entities or cross check against validated data set
- data enhancement like N, E, W, S to North, East, West, South
- changing reference data - 2 digit country code to 3 digit country code

# PROBLEM SET START HERE

In this problem set you work with cities infobox data, audit it, come up with a
cleaning idea and then clean it up. In the first exercise we want you to audit
the datatypes that can be found in some particular fields in the dataset.
The possible types of values can be:
- NoneType if the value is a string "NULL" or an empty string ""
- list, if the value starts with "{"
- int, if the value can be cast to int
- float, if the value can be cast to float, but CANNOT be cast to int.
   For example, '3.23e+07' should be considered a float because it can be cast
   as float but int('3.23e+07') will throw a ValueError
- 'str', for all other values

The audit_file function should return a dictionary containing fieldnames and a 
SET of the types that can be found in the field. e.g.
{"field1": set([type(float()), type(int()), type(str())]),
 "field2": set([type(str())]),
  ....
}
The type() function returns a type object describing the argument given to the 
function. You can also use examples of objects to create type objects, e.g.
type(1.1) for a float: see the test function below for examples.

Note that the first three rows (after the header row) in the cities.csv file
are not actual data points. The contents of these rows should note be included
when processing data types. Be sure to include functionality in your code to
skip over or detect these rows.

In [15]:
import codecs
import csv
import json
import pprint

def audit_file(file_name, fields):
    field_types = {}
    for field in fields:
        field_types[field] = set()

    f = open(file_name, "r")
    reader = csv.DictReader(f)
    for line in reader:
        # print line
        for field in fields:
            field_value = line[field].strip()

            # if field == "name" or field == "areaLand":
            #     print "field {} has value {}".format(field, field_value)

            if field_value == "NULL" or field_value == "":
                field_types[field].add(type(None))
                continue

            if field_value.startswith("{"):
                field_types[field].add(type([]))

            try:
                int(field_value)
                field_types[field].add(type(1))
            except:
                try:
                    float(field_value)
                    field_types[field].add(type(1.1))
                except:
                    pass

    pprint.pprint(field_types)

    return field_types

In [16]:
FIELDS = ["name", "timeZone_label", "utcOffset", "homepage", "governmentType_label",
          "isPartOf_label", "areaCode", "populationTotal", "elevation",
          "maximumElevation", "minimumElevation", "populationDensity",
          "wgs84_pos#lat", "wgs84_pos#long", "areaLand", "areaMetro", "areaUrban"]

def test1(cities):
    field_types = audit_file(cities, FIELDS)

    assert field_types["areaLand"] == {type(1.1), type([]), type(None)}
    assert field_types['areaMetro'] == {type(1.1), type(None)}

In [17]:
test1('cities.csv')

{'areaCode': set([<type 'int'>, <type 'NoneType'>]),
 'areaLand': set([<type 'float'>, <type 'list'>, <type 'NoneType'>]),
 'areaMetro': set([<type 'float'>, <type 'NoneType'>]),
 'areaUrban': set([<type 'float'>, <type 'NoneType'>]),
 'elevation': set([<type 'float'>, <type 'list'>, <type 'NoneType'>]),
 'governmentType_label': set([<type 'NoneType'>]),
 'homepage': set([<type 'NoneType'>]),
 'isPartOf_label': set([<type 'list'>, <type 'NoneType'>]),
 'maximumElevation': set([<type 'NoneType'>]),
 'minimumElevation': set([<type 'NoneType'>]),
 'name': set([<type 'list'>, <type 'NoneType'>]),
 'populationDensity': set([<type 'float'>, <type 'list'>, <type 'NoneType'>]),
 'populationTotal': set([<type 'int'>, <type 'NoneType'>]),
 'timeZone_label': set([<type 'NoneType'>]),
 'utcOffset': set([<type 'int'>, <type 'list'>, <type 'NoneType'>]),
 'wgs84_pos#lat': set([<type 'float'>]),
 'wgs84_pos#long': set([<type 'float'>])}


Since in the previous quiz you made a decision on which value to keep for the
"areaLand" field, you now know what has to be done.

Finish the function fix_area(). It will receive a string as an input, and it
has to return a float representing the value of the area or None.
You have to change the function fix_area. You can use extra functions if you
like, but changes to process_file will not be taken into account.
The rest of the code is just an example on how this function can be used.

In [18]:
def fix_area(area):

    result = area

    if area == "NULL" or area == "":
        result = None
    elif area.startswith("{"):
        areas = area.strip("{}").split("|")
        # area =
        # area1 = float(areas[1])
        result = float(areas[0])

    print "for {} output is {}".format(area, result)

    return result

In the previous quiz you recognized that the "name" value can be an array (or
list in Python terms). It would make it easier to process and query the data
later if all values for the name are in a Python list, instead of being
just a string separated with special characters, like now.

Finish the function fix_name(). It will recieve a string as an input, and it
will return a list of all the names. If there is only one name, the list will
have only one item in it; if the name is "NULL", the list should be empty.
The rest of the code is just an example on how this function can be used.

In [19]:
def fix_name(name):

    result = name
    if name.startswith("{"):
        result = name.strip("{}").split("|")
    else:
        result = [name]

    return result

In [20]:
def process_file_fun(filename, fun):
    data = []

    with open(filename, "r") as f:
        reader = csv.DictReader(f)

        #skipping the extra metadata
        for i in range(3):
            l = reader.next()

        # processing file
        for line in reader:
            fun(line)
            
            #pprint.pprint(line)
            
            data.append(line)

    return data

In [21]:
def fixArea(line):
    if "areaLand" in line:
        line["areaLand"] = fix_area(line["areaLand"])

def fixName(line):
    if "name" in line:
        line["name"] = fix_name(line["name"])

In [22]:
def test2(cities):
    data = process_file_fun(cities, fixArea)

    print "Printing three example results:"
    for n in range(5, 8):
        pprint.pprint(data[n]["areaLand"])
        
    pprint.pprint(data[3])

    assert data[3]["areaLand"] == None
    assert data[8]["areaLand"] == 55166700.0
    assert data[20]["areaLand"] == 14580000.0

In [23]:
test2('cities.csv')

for NULL output is None
for NULL output is None
for NULL output is None
for NULL output is None
for NULL output is None
for NULL output is None
for {1.01787e+08|1.019e+08} output is 101787000.0
for {3.15979e+07|3.17e+07} output is 31597900.0
for {5.51667e+07|5.53e+07} output is 55166700.0
for {6.36e+07|6.37137e+07} output is 63600000.0
for {3.78138e+07|3.79e+07} output is 37813800.0
for {2.02e+07|2.02019e+07} output is 20200000.0
for {2.87489e+08|2.875e+08} output is 287489000.0
for {2.5355e+07|2.5356e+07} output is 25355000.0
for {2.512e+08|2.51229e+08} output is 251200000.0
for {8.25e+07|8.26206e+07} output is 82500000.0
for 3.58195e+07 output is 3.58195e+07
for 1.13e+07 output is 1.13e+07
for 5.32e+07 output is 5.32e+07
for {4.48e+06|4.48068e+06} output is 4480000.0
for {1.458e+07|1.45816e+07} output is 14580000.0
for {1.71198e+07|1.712e+07} output is 17119800.0
for {2.07e+06|2.07199e+06} output is 2070000.0
for {4.61e+06|4.61018e+06} output is 4610000.0
for {3.26e+06|3.26339e+06} o

In [24]:
def test3(cities):
    data = process_file_fun(cities, fixName)

    #print "Printing 20 results:"
    #for n in range(20):
    #    pprint.pprint(data[n]["name"])
    #pprint.pprint("data[14] is {}".format(data[14]))

    assert data[14]["name"] == ['Negtemiut', 'Nightmute']
    assert data[9]["name"] == ['Pell City Alabama']
    assert data[3]["name"] == ['Kumhari']

In [25]:
test3("cities.csv")

If you look at the full city data, you will notice that there are couple of
values that seem to provide the same information in different formats: "point"
seems to be the combination of "wgs84_pos#lat" and "wgs84_pos#long". However,
we do not know if that is the case and should check if they are equivalent.

Finish the function check_loc(). It will recieve 3 strings: first, the combined
value of "point" followed by the separate "wgs84_pos#" values. You have to
extract the lat and long values from the "point" argument and compare them to
the "wgs84_pos# values, returning True or False.

Note that you do not have to fix the values, only determine if they are
consistent. To fix them in this case you would need more information. Feel free
to discuss possible strategies for fixing this on the discussion forum.